### Explaining Sitenews Preprocessing


In [1]:
# Loading all sitenews. It takes a lot of time. 
# There are ./datasets/final_df.parquet - this function result

# from isswrapper.util.helpers import save_dataframe_to_parquet
# from isswrapper.loaders.async_sitenews import run_async, load_all_sitenews

# final_df = run_async(load_all_sitenews, max_connections=20)
# save_dataframe_to_parquet(final_df, os.path.join(datasets_folder_path,"final_df.parquet"))

In [2]:
from isswrapper.util.helpers import read_parquet_into_dataframe, save_dataframe_to_parquet
from isswrapper.loaders.history import security_history, run_async
import pandas as pd 
import datetime
from bs4 import BeautifulSoup
import os
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
from pnd_moex.general.plots import pnd_sample_data_plot



In [3]:
current_path = os.getcwd()
project_path = os.path.dirname(current_path)
datasets_folder_path = os.path.join(project_path, 'datasets') 
df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "final_df.parquet"))
print(df.shape)
df.head()


(40799, 6)


,id,tag,title,published_at,modified_at,body
0,63318,site,О выявленном несоответствии ценных бумаг,2023-08-21 19:00:00,2023-08-21 19:00:05,<p>В соответствии с Правилами листинга ПАО Мос...
1,63316,site,Об установлении риск-параметров на фондовом ры...,2023-08-21 18:54:24,2023-08-21 18:54:24,<p>С 22.08.2023 решением НКО НКЦ (АО) устанавл...
2,63315,site,О вступлении в силу внутренних документов Моск...,2023-08-21 18:53:22,2023-08-21 18:53:23,"<p><span>Информируем, что 28 августа 2023 года..."
3,63314,site,Об изменении уровня листинга ценных бумаг,2023-08-21 18:24:25,2023-08-21 18:24:26,<p>В соответствии с Правилами листинга ПАО Мос...
4,63313,site,О регистрации программ биржевых облигаций,2023-08-21 18:19:22,2023-08-21 18:19:22,<p><span>В соответствии c Правилами листинга П...


We're dealing with a bunch of news articles, but our focus is on news related to unusual behavior in securities. We've noticed that if an article contains the phrase 'отклонения цен заявок' (price order deviations), it includes tables with information about changes in risk parameters for certain securities. What does that imply? Essentially, it means there might be fewer restrictions or, on the flip side, new limitations introduced. Our main interest lies in the latter scenario. Here, it's either something positive that's happened, or it could indicate someone with questionable intentions trying a pump-and-dump strategy.

So, our initial step is to sort through all the records that match our criteria.

In [4]:
filtered_df= df[df["body"].str.contains("отклонения цен заявок")]
filtered_df.shape

(140, 6)

In [5]:
def has_table(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    return table is not None

w_table_df = filtered_df[filtered_df["body"].apply(has_table)]
w_table_df.shape

(140, 6)

The next phase involves extracting these tables from the news articles and trimming away extraneous details.

However, not all tables within this dataset are well-organized. The older ones might lack the 'th' tag. In these instances, we will rely on the first row to act as the header.

The 'extract_table' function is purpose-built for this task. It functions by taking the HTML body and parsing out the table, converting it into a pandas DataFrame.

Subsequently, the 'preprocess' function trims away redundant and uninformative particulars—such as full names and numbers. Following this, it rearranges the column names for enhanced convenience and converts percentage data into standard numerical values.

In [6]:
def extract_table(html):
    soup = BeautifulSoup(html, "html.parser")
    header = None
    if not soup.findAll("th"):
        header = 0  
    tables = pd.read_html(str(soup), header=header)
    table = tables[0]
    return table

def preprocess(df):
    if df.shape[1] == 5:
        df.drop(columns = "№", inplace=True)
    df.drop(columns="Название", inplace=True)
    df.rename(
        columns ={df.columns[0]:"token", df.columns[1]:"current_limit", df.columns[2]:"new_limit"},
        inplace=True,
        )
    df["current_limit"] = df["current_limit"].str.rstrip("%").astype(float)
    df["new_limit"] = df["new_limit"].str.rstrip("%").astype(float)
    return df

w_table_df["table"] = w_table_df["body"].apply(extract_table)
w_table_df["t_shape"] = w_table_df["table"].apply(lambda x: x.shape[1])
w_table_df.drop(w_table_df[w_table_df["t_shape"]>6].index, inplace=True)
w_table_df["table"] = w_table_df["table"].apply(preprocess)
w_table_df.shape


(136, 8)

Why do we omit tables with more than 6 columns? In short, this concerns just 3 entries among the entire dataset, and these tables don't hold significant data. Moreover, automatic data extraction becomes notably challenging due to their lack of standardization. Thus, it proves more convenient to bypass such tables altogether.


In [7]:
pd_list = []
for _, row in w_table_df.iterrows():
    table = row["table"]
    table["p_date"] = row["published_at"]
    pd_list.append(table)
f_df = pd.concat(pd_list).reset_index(drop=True)
f_df.sample(5)

,token,current_limit,new_limit,p_date
100,RTSBP,40.0,10.0,2021-09-22 20:09:00
154,VSYDP,40.0,10.0,2020-04-27 20:19:00
57,RKKE,10.0,40.0,2023-02-08 16:52:00
22,VRSB,40.0,10.0,2023-07-19 22:00:00
138,UBER-RM,40.0,40.0,2020-11-30 15:43:00


Now, let's direct our attention to the entries where new limitations have been introduced (new_limit < current_limit).

In [8]:
f_df = f_df[f_df["current_limit"]>f_df["new_limit"]]
f_df.shape

(95, 4)

At this juncture, we have identified the securities that warrant scrutiny for potential pump-and-dump activities.

In [9]:
pnd_df = pd.DataFrame(f_df.groupby(by = "token")["p_date"].apply(list)).reset_index()
print(pnd_df.shape)
pnd_df.head()

(72, 2)


,token,p_date
0,ABRD,[2020-08-06 19:14:00]
1,ALBK,[2021-08-09 19:19:00]
2,ASSB,[2023-08-08 19:30:00]
3,BELU,[2021-02-18 20:12:00]
4,BLNG,[2023-02-09 08:59:00]


In [9]:
save_dataframe_to_parquet(pnd_df, os.path.join(datasets_folder_path, "pnd_token_date.parquet"))

The subsequent stage involves gathering comprehensive historical data for these securities. For convenience, I will consolidate all the data into a single dataframe.

In [10]:
token_ts_list = []
for token in pnd_df["token"].values:
    res = run_async(security_history, sec_id = token)
    token_ts_list.append(res)
ts_df = pd.concat(token_ts_list)
ts_df.head()


,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,WAPRICE,CLOSE,VOLUME,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR,currencyid
0,TQBR,2014-06-09,АбрауДюрсо,ABRD,32.0,586611.0,132.72,132.70,134.05,133.62,133.32,133.30,4400.0,133.32,133.32,133.32,586611.0,586611.0,586611.0,NaN,3,SUR,-1.41,RUB
1,TQBR,2014-06-10,АбрауДюрсо,ABRD,9.0,145628.0,133.90,130.91,133.90,131.82,132.39,131.02,1100.0,133.13,133.18,133.13,732239.0,519414.0,732239.0,NaN,3,SUR,-1.71,RUB
2,TQBR,2014-06-11,АбрауДюрсо,ABRD,42.0,937910.0,132.50,130.50,133.89,133.17,132.10,133.89,7100.0,132.10,132.10,132.1,937910.0,937910.0,937910.0,NaN,3,SUR,2.19,RUB
3,TQBR,2014-06-16,АбрауДюрсо,ABRD,39.0,772449.0,133.70,132.17,133.86,132.17,133.18,132.17,5800.0,133.18,133.18,133.18,772449.0,772449.0,772449.0,NaN,3,SUR,-1.28,RUB
4,TQBR,2014-06-17,АбрауДюрсо,ABRD,14.0,223421.0,131.88,131.11,131.88,131.11,131.42,131.11,1700.0,NaN,132.17,NaN,0.0,502260.0,0.0,NaN,3,SUR,-0.80,RUB


In [11]:
save_dataframe_to_parquet(ts_df, os.path.join(datasets_folder_path,"time_series_securities_pnd.parquet"))

Now let's take a look 

In [11]:
# ts_df = read_parquet_into_dataframe(os.path.join(datasets_folder_path, "time_series_securities_pnd.parquet"))

pnd_sample_data_plot(pnd_df, ts_df)